In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import cv2
from PIL import Image,ImageOps
from PIL import ImageDraw 
import shutil
import os
from PIL import Image
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array
from keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Conv2D, Concatenate, Flatten, Add, Dropout, GRU
import random
import datetime
from nltk.translate.bleu_score import sentence_bleu

In [2]:
train = pd.read_csv("train_df_final.csv",nrows=1000)

In [3]:
train.head()

,Unnamed: 0,Unnamed: 0.1,patient_id,image1_id,image2_id,report
0,471,471,CXR1505_IM-0330,CXR1505_IM-0330-1001,CXR1505_IM-0330-2001,startseq normal heart size clear lungs no pneu...
1,9,9,CXR1007_IM-0008,CXR1007_IM-0008-1001,CXR1007_IM-0008-2001,startseq trachea is midline the cardiomediasti...
2,1499,1499,CXR2624_IM-1111,CXR2624_IM-1111-1001,CXR2624_IM-1111-2001,startseq heart size and mediastinal contour ar...
3,54,54,CXR1054_IM-0040,CXR1054_IM-0040-1001,CXR1054_IM-0040-1002,startseq heart size is normal stable mediastin...
4,1411,1411,CXR2529_IM-1044,CXR2529_IM-1044-1001,CXR2529_IM-1044-2001,startseq the lungs are clear bilaterally speci...


In [4]:
test = pd.read_csv("test_df_final.csv",nrows=200)

In [5]:
test.head()

,Unnamed: 0,Unnamed: 0.1,patient_id,image1_id,image2_id,report
0,11,11,CXR1009_IM-0010,CXR1009_IM-0010-1001,CXR1009_IM-0010-2001,startseq the cardiomediastinal silhouette and ...
1,23,23,CXR1020_IM-0017,CXR1020_IM-0017-1001,CXR1020_IM-0017-2001,startseq lung volumes are low no focal infiltr...
2,24,24,CXR1022_IM-0017,CXR1022_IM-0017-1001,CXR1022_IM-0017-2001,startseq the heart and lungs have in the int...
3,25,25,CXR1023_IM-0018,CXR1023_IM-0018-1001,CXR1023_IM-0018-2001,startseq the cardiac silhouette and mediastinu...
4,28,28,CXR1026_IM-0021,CXR1026_IM-0021-2002,CXR1026_IM-0021-2002,startseq the lungs appear clear the heart and ...


In [6]:
train.drop('Unnamed: 0.1',axis='columns',inplace=True)
test.drop('Unnamed: 0.1',axis='columns',inplace=True)

In [7]:
train.drop('Unnamed: 0',axis='columns',inplace=True)
test.drop('Unnamed: 0',axis='columns',inplace=True)

In [8]:
train.head()

,patient_id,image1_id,image2_id,report
0,CXR1505_IM-0330,CXR1505_IM-0330-1001,CXR1505_IM-0330-2001,startseq normal heart size clear lungs no pneu...
1,CXR1007_IM-0008,CXR1007_IM-0008-1001,CXR1007_IM-0008-2001,startseq trachea is midline the cardiomediasti...
2,CXR2624_IM-1111,CXR2624_IM-1111-1001,CXR2624_IM-1111-2001,startseq heart size and mediastinal contour ar...
3,CXR1054_IM-0040,CXR1054_IM-0040-1001,CXR1054_IM-0040-1002,startseq heart size is normal stable mediastin...
4,CXR2529_IM-1044,CXR2529_IM-1044-1001,CXR2529_IM-1044-2001,startseq the lungs are clear bilaterally speci...


In [9]:
test.head()

,patient_id,image1_id,image2_id,report
0,CXR1009_IM-0010,CXR1009_IM-0010-1001,CXR1009_IM-0010-2001,startseq the cardiomediastinal silhouette and ...
1,CXR1020_IM-0017,CXR1020_IM-0017-1001,CXR1020_IM-0017-2001,startseq lung volumes are low no focal infiltr...
2,CXR1022_IM-0017,CXR1022_IM-0017-1001,CXR1022_IM-0017-2001,startseq the heart and lungs have in the int...
3,CXR1023_IM-0018,CXR1023_IM-0018-1001,CXR1023_IM-0018-2001,startseq the cardiac silhouette and mediastinu...
4,CXR1026_IM-0021,CXR1026_IM-0021-2002,CXR1026_IM-0021-2002,startseq the lungs appear clear the heart and ...


In [10]:
train.shape, test.shape

((1000, 4), (200, 4))

In [11]:
X_train, X_test = train['patient_id'], test['patient_id']
y_train, y_test = train['report'], test['report']

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1000,), (1000,), (200,), (200,))

In [13]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(y_train.values)

In [14]:
with open('train_features_list.pkl', 'rb') as f:
    train_feat_list = pickle.load(f)

In [15]:
with open('test_features_list.pkl', 'rb') as f:
    test_feat_list = pickle.load(f)

In [16]:
len(train_feat_list)

1000

In [17]:
len(test_feat_list)

200

In [18]:
f = open('C:/Users/INDIRA/Downloads/data rescon/glove.840B.300d.pkl','rb') # 300d glove vectors  
glove_vectors = pickle.load(f)
f.close()

In [19]:
vocab_size = len(tokenizer.word_index.keys()) + 1

In [20]:
embedding_matrix = np.zeros((vocab_size,300))
for word, i in tokenizer.word_index.items():
    if word in glove_vectors.keys():
        vec = glove_vectors[word]
        embedding_matrix[i] = vec
    else:
        continue

In [21]:
batch_size = 15

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_feat_list,y_train))
train_dataset = train_dataset.shuffle(300).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_feat_list,y_test))
test_dataset = test_dataset.shuffle(300).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [24]:
def load_data(images,reports_unpadded):
    imgs = []
    input_reports = []
    output_reports = []
    for i in range(len(images)):
        sequence=[] 
        for rep in reports_unpadded[i].split(" "):
            if rep in tokenizer.word_index.keys():
                sequence.append(tokenizer.word_index[rep])
        for j in range(1,len(sequence)):
            in_seq = sequence[:j]          
            out_seq = sequence[j]            
            out_seq = tf.keras.utils.to_categorical(out_seq, num_classes=vocab_size)
            imgs.append(images[i])            
            input_reports.append(in_seq)            
            output_reports.append(out_seq)
        
    return np.array(imgs), np.array(input_reports), np.array(output_reports)

In [25]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='auto')
def maskedLoss(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    loss_ = loss_function(y_true, y_pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ = loss_*mask
    loss_ = tf.reduce_mean(loss_)
    return loss_

In [26]:
max_len=80

In [27]:
input_1=Input(shape=(2048),name="Images")
encoder_out=Dense(512,activation="relu",name="enc_dense")(input_1)


#decoder model
input_text=Input(shape=(max_len),name="text")

embedding_out=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=300,input_length=max_len,mask_zero=True,trainable=False,weights=[embedding_matrix])(input_text)

lstm_out= LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), return_sequences=True, name="LSTM1")(embedding_out)

lstm_out= LSTM(units=512, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")(lstm_out) 

x=Dropout(0.35)(lstm_out)
add=tf.keras.layers.Add()([encoder_out, x])
  
x=Dense(512,kernel_initializer=tf.keras.initializers.he_normal(seed=1),activation="relu")(add)

x1=Dropout(0.25)(x)

x1=Dense(vocab_size,activation="softmax")(x1)
#encoder_decoder_model
encoder_decoder=Model(inputs=[input_1,input_text],outputs=x1)
encoder_decoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 80, 300)      330600      text[0][0]                       
__________________________________________________________________________________________________
LSTM1 (LSTM)                    (None, 80, 256)      570368      embedding[0][0]                  
__________________________________________________________________________________________________
Images (InputLayer)             [(None, 2048)]       0                                            
______________________________________________________________________________________________

In [28]:
encoder_decoder.compile(optimizer="Adam", loss = maskedLoss)

In [29]:
train_loss_list = []

for epoch in range(10):
    print('EPOCH : ',epoch+1)
    batch_loss_train = 0
    
    for img, report in train_dataset:        
        r1 = [word.decode('utf-8') for word in np.array(report)]  
        img_input, rep_input, output_word = load_data(img.numpy(), r1)
        rep_input = tf.keras.preprocessing.sequence.pad_sequences(rep_input, maxlen=80, padding='post')
        img_input=tf.reshape(img_input,shape=(img_input.shape[0],img_input.shape[-1]))        
        loss = encoder_decoder.train_on_batch([img_input, rep_input], output_word)        
        batch_loss_train += loss

    train_loss = batch_loss_train/(len(y_train)//15)
    print('Training Loss: {}'.format(train_loss))
    
encoder_decoder.save_weights('encoder_decoder_epoch_'+ str(epoch+1) + '.h5')

EPOCH :  1


<ipython-input-24-9aa7dabe9026>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(imgs), np.array(input_reports), np.array(output_reports)


Training Loss: 0.004779368519049251
EPOCH :  2
Training Loss: 0.004259754614340085
EPOCH :  3
Training Loss: 0.0038818792320054136
EPOCH :  4
Training Loss: 0.0034573689018460836
EPOCH :  5
Training Loss: 0.003014479336949686
EPOCH :  6
Training Loss: 0.0026333709176855555
EPOCH :  7
Training Loss: 0.002369077343521922
EPOCH :  8
Training Loss: 0.002172225320739954
EPOCH :  9
Training Loss: 0.002011224671591525
EPOCH :  10
Training Loss: 0.0018691809876172831


In [35]:
def img_preprocess(img):
    img=img_to_array(img)
    img=preprocess_input(img)
    img=cv2.resize(img,(224,224))
    img=img/255.0
    img=np.expand_dims(img, axis=0)
    return img

In [33]:
from tensorflow.keras.applications import DenseNet121

image_shape= (224,224,3)
mod2=DenseNet121(include_top=False,input_shape=image_shape,pooling="avg")
las2=Dense(14,"sigmoid")(mod2.output)

mod2=Model(inputs=mod2.input,outputs=las2)
mod2.load_weights("c:/Users/INDIRA/Downloads/data rescon/chexnet_weights.h5")

In [34]:
final_chexnet_model=Model(inputs=mod2.inputs,outputs=mod2.layers[-2].output,name="Chexnet_model")

In [36]:
image_1= tf.keras.Input(shape=(224,224,3),name="image_1_features")
image_2= tf.keras.Input(shape=(224,224,3),name="image_2_features")
image_1_out=final_chexnet_model(image_1)
image_2_out=final_chexnet_model(image_2)
conc=tf.keras.layers.Concatenate(axis=-1)([image_1_out,image_2_out])
feature_extraction_model=Model(inputs=[image_1,image_2],outputs=conc)

In [112]:
def evaluation(image1,image2):
    image_features=feature_extraction_model([image1,image2])
    output_report=''
    input_rep= 'startseq'
    image_features=tf.reshape(image_features,shape=(-1,image_features.shape[-1]))
    for i in range(max_len):
        input_tokens = [tokenizer.word_index[w] for w in input_rep.split()]
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_tokens],max_len, padding='post')
        results = encoder_decoder.predict([image_features,input_padded])
        arg = np.argmax(results[0]) 
        if tokenizer.index_word[arg]=='endseq':
            output_report+=tokenizer.index_word[arg]+" "
            break
        else:
            input_rep = input_rep + ' ' + tokenizer.index_word[arg]
            output_report = output_report+tokenizer.index_word[arg]+" "
    return output_report

In [109]:
img1=Image.open('C:/Users/INDIRA/Downloads/data rescon/images/CXR1009_IM-0010-1001'+'.png')
img2=Image.open('C:/Users/INDIRA/Downloads/data rescon/images/CXR1009_IM-0010-2001'+'.png')
img1=img_preprocess(img1)
img2=img_preprocess(img2)
result=evaluation(img1,img2) 
actual=y_test[0]
print("Actual Report: ",actual)
print("Generated Report: ",result) 
print("BLEU SCORE IS: ",sentence_bleu(actual,result))

Actual Report:  startseq the cardiomediastinal silhouette and pulmonary vasculature are within normal limits there is no pneumothorax or pleural effusion there are no focal areas of consolidation endseq
Generated Report:  the heart is normal in size the mediastinum is unremarkable the lungs are clear endseq 
BLEU SCORE IS:  1.1930496181487654e-231


In [113]:
img1=Image.open('C:/Users/INDIRA/Downloads/data rescon/images/CXR1026_IM-0021-2002'+'.png')
img2=Image.open('C:/Users/INDIRA/Downloads/data rescon/images/CXR1026_IM-0021-2002'+'.png')
img1=img_preprocess(img1)
img2=img_preprocess(img2)
result=evaluation(img1,img2) 
actual=y_test[4]
print("Actual Report: ",actual)
print("Generated Report: ",result) 
print("BLEU SCORE IS: ",sentence_bleu(actual,result))

Actual Report:  startseq the lungs appear clear the heart and pulmonary  appear normal the pleural spaces are clear mediastinal contours are normal endseq
Generated Report:  the heart is normal in size the mediastinum is unremarkable there is no pleural effusion or pneumothorax there is no focal air space opacity to suggest a pneumonia there is a calcified granuloma in the right lung base there are no acute bony findings endseq 
BLEU SCORE IS:  9.490559085273313e-232
